In [1]:
import requests
import time
import json
import mysql.connector
import pprint
import MySQLdb

from datetime import datetime, timedelta


## get repoList

In [2]:
import pickle
filepath = '/Users/ellesmac/jupyter_notebook/repo_list.pkl'
with open(filepath, 'rb') as f:
    repo_list = pickle.load(f)

In [3]:
filepathLast = '/Users/ellesmac/jupyter_notebook/last_list.pkl'
with open(filepathLast, 'rb') as f:
    li_list = pickle.load(f)

In [ ]:
finalList = li_list+repo_list
len(set(finalList))

## get repo id

In [6]:
# get repo id
repoInfo = requests.get('http://api.github.com/repos/'+finalList[0])
repoInfo_json = json.loads(repoInfo.text)
repo1 = repoInfo_json['id']

## Storage

In [37]:
# import contributor to contributor / user/ url table
import MySQLdb
listContris= requests.get('https://api.github.com/repos/'+finalList[0]+'/contributors?per_page=100',auth = ('elletsai', '0f5c16b728f522f5b19c17c8981b8b9d51619df3'))
listContris_json = json.loads(listContris.text)
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')

#--- user
userQuery_list = ['type','login','id','site_admin','gravatar_id','node_id']
userQuery_table = tuple(listContris_json[3][x] for x in userQuery_list)

# --- url
urlQuery_list = ['avatar_url','url','html_url','followers_url','following_url','gists_url','starred_url','subscriptions_url','organizations_url','repos_url','events_url','received_events_url','id']
urlQuery_table = tuple(listContris_json[3][x] for x in urlQuery_list)
#-- contributor
contriList = tuple([listContris_json[3]['id'], repo1])

sqlQuery_contri = "INSERT INTO contributor(user_id, repo_id) VALUES(%s, %s)"
sqlQuery_user= "INSERT INTO user(user_type, user_login, user_id, site_admin, user_gravatarId, user_nodeId) VALUES (%s,%s,%s,%s, %s, %s)"
sqlQuery_url= "INSERT INTO url(avatarUrl, userUrl, htmlUrl, followersUrl, followingUrl, gistsUrl, starredUrl, subscriptionsUrl, organizationsUrl, reposUrl, eventsUrl, receivedEventsUrl, user_id) VALUES (%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s)"


mycursor = conn.cursor()
try:
    mycursor.execute(sqlQuery_user, userQuery_table)
    mycursor.execute(sqlQuery_url, urlQuery_table)
        
except mysql.connector.IntegrityError as err:
    print('duplicate user')
    pass

try:
    mycursor.execute(sqlQuery_contri, contriList)
except mysql.connector.IntegrityError as err:
    print('duplicate contributor')
    pass


conn.commit()
conn.close()




In [ ]:
#import watcher
import MySQLdb

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')


userQuery_list = ['type','login','id','site_admin','gravatar_id','node_id']
urlQuery_list = ['avatar_url','url','html_url','followers_url','following_url','gists_url','starred_url','subscriptions_url','organizations_url','repos_url','events_url','received_events_url','id']

sqlQuery_watcher = "INSERT INTO watcher(watcher_id, repo_id) VALUES(%s, %s)"
sqlQuery_user= "INSERT INTO user(user_type, user_login, user_id, site_admin, user_gravatarId, user_nodeId) VALUES (%s,%s,%s,%s, %s, %s)"
sqlQuery_url= "INSERT INTO userUrl(avatarUrl, userUrl, htmlUrl, followersUrl, followingUrl, gistsUrl, starredUrl, subscriptionsUrl, organizationsUrl, reposUrl, eventsUrl, receivedEventsUrl, user_id) VALUES (%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s)"

mycursor = conn.cursor()

lis = []
page = 1
while True:
    listWatcher= requests.get('https://api.github.com/repos/'+finalList[0]+'/watchers?page='+str(page)+'&per_page=100',auth = ('elletsai', '0f5c16b728f522f5b19c17c8981b8b9d51619df3'))
    listWatcher_json = json.loads(listWatcher.text)
    
    if len(listWatcher_json) == 0:
        break
    for y in range(len(listWatcher_json)):

        userQuery_table = tuple(listWatcher_json[y][x] for x in userQuery_list)
        urlQuery_table = tuple(listWatcher_json[y][x] for x in urlQuery_list)
        watcherList = tuple([listWatcher_json[y]['id'], repo1])
        
        
        try:
            mycursor.execute(sqlQuery_user, userQuery_table)
            mycursor.execute(sqlQuery_url, urlQuery_table)

        except mysql.connector.IntegrityError as err:
            print('duplicate user')
            pass

        try:
            mycursor.execute(sqlQuery_watcher, watcherList)
        except mysql.connector.IntegrityError as err:
            print('duplicate watcher')
            pass

        conn.commit()
        
    print(page)
    page += 1
        
        

conn.close()


In [32]:
# import branch
import MySQLdb
listBranch= requests.get('https://api.github.com/repos/'+finalList[0]+'/branches')
listBranch_json = json.loads(listBranch.text)
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')

sqlQuery_branch = "INSERT INTO branch(branch_commitSha,branch_commitUrl,branch_name,branch_protected, repo_id) VALUES(%s,%s,%s,%s,%s)"
mycursor = conn.cursor()

for x in range(len(listBranch_json)):
    branchQuery_table = tuple([listBranch_json[x]['commit']['sha'], listBranch_json[x]['commit']['url'], listBranch_json[x]['name'], listBranch_json[x]['protected'], repo1])

    mycursor.execute(sqlQuery_branch, branchQuery_table)
    conn.commit()

    

conn.close()
    
    
    

In [ ]:
#print db data
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')
mycursor = conn.cursor()
mycursor.execute("SELECT *FROM watcher")
print(mycursor.fetchall())
#conn.close()


In [13]:
# import repo to repo table
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')
#    DHE-RSA-AES256-SHA

repoInfo = requests.get('http://api.github.com/repos/'+finalList[0])
repoInfo_json = json.loads(repoInfo.text)
sqlQuery_repo= "INSERT INTO repo(repo_archived, repo_created_at, repo_default_branch, repo_description, repo_disabled, repo_fork, \
                                repo_forks,  repo_forks_count, repo_full_name, repo_has_downloads, repo_has_issues, repo_has_pages, \
                                repo_has_projects,repo_has_wiki,repo_homepage, repo_id, repo_language, repo_name, repo_network_count,\
                                repo_open_issues, repo_open_issues_count, repo_private, repo_size, repo_stargazers_count,\
                                repo_subscribers_count, repo_watchers, repo_watchers_count) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,\
                                %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
repo_queryList = ['archived', 'created_at', 'default_branch', 'description', 'disabled', 'fork', 'forks', 'forks_count', 'full_name', \
                  'has_downloads', 'has_issues', 'has_pages', 'has_projects', 'has_wiki', 'homepage', 'id', 'language', 'name', 'network_count',\
                  'open_issues', 'open_issues_count', 'private', 'size', 'stargazers_count', 'subscribers_count', 'watchers', 'watchers_count']

repo_queryTable = tuple(repoInfo_json[x] for x in repo_queryList)

mycursor = conn.cursor()
mycursor.execute("SET SESSION MAX_EXECUTION_TIME=1000")
mycursor.execute(sqlQuery_repo, repo_queryTable)

conn.commit()

conn.close()



In [57]:
# import license and repoUrl
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')


repoInfo = requests.get('http://api.github.com/repos/'+finalList[0])
repoInfo_json = json.loads(repoInfo.text)
# ==== license
sqlQuery_license = "INSERT INTO license(license_node_id, license_key, license_name, license_spdx_id, license_url, repo_id) VALUES (%s,%s,%s,%s,%s,%s)"
licenseQuery_list = ['node_id','key', 'name', 'spdx_id', 'url']
licenseQuery_table = tuple(repoInfo_json['license'][x] for x in licenseQuery_list)
licenseQuery_table += (repo1,)

# ===== repoUrl
sqlQuery_repoUrl = "INSERT INTO repoUrl(archive_url, assignees_url, blobs_url, branches_url, clone_url, collaborators_url, comments_url,\
                    commits_url, compare_url, contents_url, contributors_url, deployments_url, events_url, forks_url, git_commits_url, git_tags_url,\
                    git_url, hooks_url, html_url, issue_comment_url, issue_events_url, keys_url, labels_url, languages_url, merges_url, milestones_url,\
                    notifications_url, pulls_url, ssh_url, stargazers_url, statuses_url, subscription_url, svn_url, tags_url, teams_url, trees_url, repo_url, repo_id)\
                    VALUES("+str('%s'+',%s'*37)+")"

repoUrlQuery_list = ['archive_url', 'assignees_url', 'blobs_url', 'branches_url', 'clone_url', 'collaborators_url', 'comments_url','commits_url', 'compare_url', 'contents_url', 
                'contributors_url', 'deployments_url', 'events_url', 'forks_url', 'git_commits_url', 'git_tags_url','git_url', 'hooks_url', 'html_url', 'issue_comment_url',
                'issue_events_url', 'keys_url', 'labels_url', 'languages_url', 'merges_url', 'milestones_url', 'notifications_url', 'pulls_url', 'ssh_url', 'stargazers_url',
                'statuses_url', 'subscription_url', 'svn_url', 'tags_url', 'teams_url', 'trees_url', 'url']

repoUrlQuery_table = tuple(repoInfo_json[x] for x in repoUrlQuery_list)

repoUrlQuery_table += (repo1,)

mycursor = conn.cursor()
mycursor.execute(sqlQuery_license, licenseQuery_table)
mycursor.execute(sqlQuery_repoUrl, repoUrlQuery_table)

conn.commit()

conn.close()




In [ ]:
# import fork

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')

mycursor = conn.cursor()
#sql Query commend

sqlQuery_fork = "INSERT INTO fork(fork_id, fork_node_id, fork_name, fork_private, fork_description, fork, fork_created_at, fork_updated_at, fork_pushed_at, fork_homepage, fork_size, fork_stargazers_count, fork_watchers_count, fork_language,\
                fork_has_issues, fork_has_projects, fork_has_downloads, fork_has_wiki, fork_has_pages, fork_forks_count, fork_archived, fork_disabled, fork_open_issues_count, fork_forks, fork_open_issues, fork_watchers, fork_default_branch,\
                fork_permissions_admin, fork_permissions_push, fork_permissions_pull, fork_owner_id, repo_id) VALUES("+str('%s'+',%s'*31)+")"

sqlQuery_forkUrl = "INSERT INTO forkUrl(fork_html_url, fork_url, fork_forks_url, fork_keys_url, fork_collaborators_url, fork_teams_url, fork_hooks_url, fork_issue_events_url, fork_events_url, fork_assignees_url, fork_branches_url, fork_tags_url, fork_blobs_url, \
                    fork_git_tags_url, fork_git_refs_url, fork_trees_url, fork_statuses_url, fork_languages_url, fork_stargazers_url, fork_contributors_url, fork_subscribers_url, fork_subscription_url, fork_commits_url, fork_git_commits_url, fork_comments_url, \
                    fork_issue_comment_url, fork_contents_url, fork_compare_url, fork_merges_url, fork_archive_url, fork_downloads_url, fork_issues_url, fork_pulls_url, fork_milestones_url, fork_notifications_url, fork_labels_url, fork_releases_url, fork_deployments_url,\
                    fork_git_url, fork_ssh_url, fork_clone_url, fork_svn_url, fork_id) VALUES("+str('%s'+',%s'*42)+")"

sqlQuery_forkLicense = "INSERT INTO forkLicense(forkLicense_key, forkLicense_name, forkLicense_spdx_id, forkLicense_url, forkLicense_node_id, fork_id) VALUES("+str('%s'+',%s'*5)+")"

sqlQuery_user= "INSERT INTO user(user_type, user_login, user_id, site_admin, user_gravatarId, user_nodeId) VALUES (%s,%s,%s,%s, %s, %s)"
sqlQuery_url= "INSERT INTO userUrl(avatarUrl, userUrl, htmlUrl, followersUrl, followingUrl, gistsUrl, starredUrl, subscriptionsUrl, organizationsUrl, reposUrl, eventsUrl, receivedEventsUrl, user_id) VALUES (%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s)"

# Query list to get data from json
userQuery_list = ['type','login','id','site_admin','gravatar_id','node_id']
urlQuery_list = ['avatar_url','url','html_url','followers_url','following_url','gists_url','starred_url','subscriptions_url','organizations_url','repos_url','events_url','received_events_url','id']
forkQuery_list = ['id','node_id', 'name', 'private', 'description', 'fork', 'created_at', 'updated_at', 'pushed_at', 'homepage', 'size', 'stargazers_count', 'watchers_count','language','has_issues',
                 'has_projects', 'has_downloads', 'has_wiki', 'has_pages', 'forks_count', 'archived', 'disabled', 'open_issues_count','forks', 'open_issues_count','watchers', 'default_branch']

forkUrlQuery_list = ['html_url', 'url', 'forks_url', 'keys_url', 'collaborators_url', 'teams_url', 'hooks_url', 'issue_events_url', 'events_url', 'assignees_url', 'branches_url', 'tags_url', 'blobs_url','git_tags_url', 'git_refs_url',
                     'trees_url', 'statuses_url', 'languages_url', 'stargazers_url', 'contributors_url', 'subscribers_url', 'subscription_url', 'commits_url', 'git_commits_url', 'comments_url', 'issue_comment_url', 'contents_url',
                     'compare_url', 'merges_url', 'archive_url', 'downloads_url', 'issues_url', 'pulls_url', 'milestones_url', 'notifications_url', 'labels_url', 'releases_url', 'deployments_url',
                     'git_url', 'ssh_url', 'clone_url', 'svn_url','id']
forkLicenseQuery_list = ['key', 'name', 'spdx_id', 'url', 'node_id']


forkpage = 1

while True:
    
    listFork= requests.get('https://api.github.com/repos/'+finalList[0]+'/forks?&page='+str(forkpage)+'&per_page=100',auth = ('elletsai', '0f5c16b728f522f5b19c17c8981b8b9d51619df3'))
    listFork_json = json.loads(listFork.text)
    if len(listFork_json) == 0:
        break
        
    for y in range(len(listFork_json)):
        
        userQuery_table = tuple(listFork_json[y]['owner'][x] for x in userQuery_list)
        
        urlQuery_table = tuple(listFork_json[y]['owner'][x] for x in urlQuery_list)
        
        forkQuery_table = tuple(listFork_json[y][x] for x in forkQuery_list)
        forkQuery_table += tuple([listFork_json[y]['permissions']['admin'], listFork_json[y]['permissions']['push'], listFork_json[y]['permissions']['pull'],listFork_json[y]['owner']['id'],repo1,])

        forkUrlQuery_table = tuple(listFork_json[y][x] for x in forkUrlQuery_list)
        
        forkLicenseQuery_table = tuple(listFork_json[y]['license'][x] for x in forkLicenseQuery_list)
        forkLicenseQuery_table += tuple([listFork_json[y]['id']])
   
        # import to mySQL
        try:
            mycursor.execute(sqlQuery_user, userQuery_table)
            mycursor.execute(sqlQuery_url, urlQuery_table)

        except mysql.connector.IntegrityError as err:
            pass

        try:
            mycursor.execute(sqlQuery_fork, forkQuery_table)
            mycursor.execute(sqlQuery_forkUrl, forkUrlQuery_table)
            mycursor.execute(sqlQuery_forkLicense, forkLicenseQuery_table)
        except mysql.connector.IntegrityError as err:
            pass
        conn.commit()
        
    print(forkpage)
    forkpage += 1
    

conn.close()

In [ ]:
# store repo org and owner to user

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')

# ===== org
repoInfo = requests.get('http://api.github.com/repos/'+finalList[0])
repoInfo_json = json.loads(repoInfo.text)
userQuery_list = ['type','login','id','site_admin','gravatar_id','node_id']
userQuery_repoOrg_table = tuple(repoInfo_json['organization'][x] for x in userQuery_list)


urlQuery_list = ['avatar_url','url','html_url','followers_url','following_url','gists_url','starred_url','subscriptions_url','organizations_url','repos_url','events_url','received_events_url','id']
urlQuery_repoOrg_table = tuple(repoInfo_json['organization'][x] for x in urlQuery_list)
sqlQuery_url= "INSERT INTO url(avatarUrl, userUrl, htmlUrl, followersUrl, followingUrl, gistsUrl, starredUrl, subscriptionsUrl, organizationsUrl, reposUrl, eventsUrl, receivedEventsUrl, user_id) VALUES (%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s)"
sqlQuery_user= "INSERT INTO user(user_type, user_login, user_id, site_admin, user_gravatarId, user_nodeId) VALUES (%s,%s,%s,%s, %s, %s)"

mycursor = conn.cursor()
mycursor.execute(sqlQuery_user, userQuery_repoOrg_table)
mycursor.execute(sqlQuery_url, urlQuery_repoOrg_table)

#==== owner
userQuery_repoOwner_table = tuple(repoInfo_json['owner'][x] for x in userQuery_list)
urlQuery_repoOwner_table = tuple(repoInfo_json['owner'][x] for x in urlQuery_list)

mycursor.execute(sqlQuery_user, userQuery_repoOwner_table)
mycursor.execute(sqlQuery_url, urlQuery_repoOwner_table)

conn.commit()
conn.close()



In [ ]:
# store owner and org when storing data in repo
# try when storing repo2
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin='mysql_native_password')
#    DHE-RSA-AES256-SHA

repoInfo = requests.get('http://api.github.com/repos/'+finalList[0])
repoInfo_json = json.loads(repoInfo.text)
sqlQuery_repo= "INSERT INTO repo(repo_archived, repo_created_at, repo_default_branch, repo_description, repo_disabled, repo_fork, \
                                repo_forks,  repo_forks_count, repo_full_name, repo_has_downloads, repo_has_issues, repo_has_pages, \
                                repo_has_projects,repo_has_wiki,repo_homepage, repo_id, repo_language, repo_name, repo_network_count,\
                                repo_open_issues, repo_open_issues_count, repo_private, repo_size, repo_stargazers_count,\
                                repo_subscribers_count, repo_watchers, repo_watchers_count, repo_org_id, repo_owner_id) VALUES("+str('%s'+',%s'*28)+")"
repo_queryList = ['archived', 'created_at', 'default_branch', 'description', 'disabled', 'fork', 'forks', 'forks_count', 'full_name', \
                  'has_downloads', 'has_issues', 'has_pages', 'has_projects', 'has_wiki', 'homepage', 'id', 'language', 'name', 'network_count',\
                  'open_issues', 'open_issues_count', 'private', 'size', 'stargazers_count', 'subscribers_count', 'watchers', 'watchers_count']

repo_queryTable = tuple(repoInfo_json[x] for x in repo_queryList)
repo_org_id = repoInfo_json['organization']['id']
repo_owner_id = repoInfo_json['owner']['id']
repo_queryTable +=  (repo_org_id,repo_owner_id,)
mycursor = conn.cursor()
mycursor.execute("SET SESSION MAX_EXECUTION_TIME=1000")
mycursor.execute(sqlQuery_repo, repo_queryTable)

conn.commit()

conn.close()
          
                     

In [ ]:
repo_queryList = ['archived', 'created_at', 'default_branch', 'description', 'disabled', 'fork', 'forks', 'forks_count', 'full_name', \
                  'has_downloads', 'has_issues', 'has_pages', 'has_projects', 'has_wiki', 'homepage', 'id', 'language', 'name', 'network_count',\
                  'open_issues', 'open_issues_count', 'private', 'size', 'stargazers_count', 'subscribers_count', 'watchers', 'watchers_count']

repo_queryTable = tuple(repoInfo_json[x] for x in repo_queryList)
repo_org_id = repoInfo_json['organization']['id']
repo_owner_id = repoInfo_json['owner']['id']
repo_queryTable +=  (repo_org_id,repo_owner_id,)
repo_queryTable

In [ ]:
# import contributors
listContris= requests.get('https://api.github.com/repos/'+finalList[0]+'/contributors?per_page=100',auth = ('elletsai', '0f5c16b728f522f5b19c17c8981b8b9d51619df3'))
listContris_json = json.loads(listContris.text)

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    port = 3306,
    password = 'lovily85',
    db = 'github',
    auth_plugin= 'mysql_native_password')
mycursor = conn.cursor()
sqlQuery_contri = "INSERT INTO contributor(user_id, repo_id) VALUES(%s, %s)"
contriList = tuple([listContris_json[0]['id'], repo1])
mycursor.execute(sqlQuery_contri, contriList)
conn.commit()
conn.close()